In [1]:
import os
import nrrd
import SimpleITK as sitk
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
def reshape_and_standardize_image(image, target_shape=(1, 3, 1482, 922)):
    # Remove the last dimension if it's 1
    if image.ndim == 4 and image.shape[-1] == 1:
        image = np.squeeze(image, axis=-1)
    
    # Ensure the input has 3 dimensions (channels, height, width)
    if image.ndim != 3:
        raise ValueError("Input image should have 3 dimensions (channels, height, width) or 4 dimensions with the last being 1")
    
    # Ensure we have 3 channels
    if image.shape[0] < 3:
        image = np.pad(image, ((0, 3 - image.shape[0]), (0, 0), (0, 0)), mode='constant', constant_values=0)
    elif image.shape[0] > 3:
        image = image[:3]
    
    # Transpose the array to (height, width, channels)
    image = np.transpose(image, (1, 2, 0))
    
    # Crop or pad height and width
    target_height, target_width = target_shape[2], target_shape[3]
    
    # Crop if larger than target
    if image.shape[0] > target_height:
        start = (image.shape[0] - target_height) // 2
        image = image[start:start+target_height, :, :]
    if image.shape[1] > target_width:
        start = (image.shape[1] - target_width) // 2
        image = image[:, start:start+target_width, :]
    
    # Pad if smaller than target
    pad_height = max(0, target_height - image.shape[0])
    pad_width = max(0, target_width - image.shape[1])
    pad_top = pad_height // 2
    pad_bottom = pad_height - pad_top
    pad_left = pad_width // 2
    pad_right = pad_width - pad_left
    
    image = np.pad(image, 
                   ((pad_top, pad_bottom), 
                    (pad_left, pad_right), 
                    (0, 0)), 
                   mode='constant', constant_values=0)
    
    # Add a new dimension at the beginning and transpose to (1, 3, height, width)
    return np.expand_dims(image, axis=0).transpose(0, 3, 1, 2)

# Example usage
original_image = np.random.rand(3, 1552, 970, 1)
standardized_image = reshape_and_standardize_image(original_image)

print("Original shape:", original_image.shape)
print("New shape:", standardized_image.shape)

Original shape: (3, 1552, 970, 1)
New shape: (1, 3, 1482, 922)


In [3]:
preName = "C:\\Users\\alexl\\Desktop\\MedCoop\\data\\cases\\"
posName = "\\i.nrrd"
dirs = [preName + i + posName for i in os.listdir(preName)]
allData = []
allCase = []
for i in dirs:

    data,_ = nrrd.read(i)
    reshaped_image = reshape_and_standardize_image(data)
    allData.append((reshaped_image,i))
    allCase.append(reshaped_image)

In [4]:
for i,j in allData:
    print(j)

C:\Users\alexl\Desktop\MedCoop\data\cases\001\i.nrrd
C:\Users\alexl\Desktop\MedCoop\data\cases\002\i.nrrd
C:\Users\alexl\Desktop\MedCoop\data\cases\003\i.nrrd
C:\Users\alexl\Desktop\MedCoop\data\cases\004\i.nrrd
C:\Users\alexl\Desktop\MedCoop\data\cases\005\i.nrrd
C:\Users\alexl\Desktop\MedCoop\data\cases\006\i.nrrd
C:\Users\alexl\Desktop\MedCoop\data\cases\009\i.nrrd
C:\Users\alexl\Desktop\MedCoop\data\cases\010\i.nrrd
C:\Users\alexl\Desktop\MedCoop\data\cases\011\i.nrrd
C:\Users\alexl\Desktop\MedCoop\data\cases\014\i.nrrd
C:\Users\alexl\Desktop\MedCoop\data\cases\015\i.nrrd
C:\Users\alexl\Desktop\MedCoop\data\cases\016\i.nrrd
C:\Users\alexl\Desktop\MedCoop\data\cases\017\i.nrrd
C:\Users\alexl\Desktop\MedCoop\data\cases\018\i.nrrd
C:\Users\alexl\Desktop\MedCoop\data\cases\024\i.nrrd
C:\Users\alexl\Desktop\MedCoop\data\cases\025\i.nrrd
C:\Users\alexl\Desktop\MedCoop\data\cases\026\i.nrrd
C:\Users\alexl\Desktop\MedCoop\data\cases\027\i.nrrd
C:\Users\alexl\Desktop\MedCoop\data\cases\028\

In [5]:
dataX = np.concatenate(allCase,axis=0)
np.save("C:\\Users\\alexl\\Desktop\\MedCoop\\code\\readed_data\\CaseInformation",dataX)


In [6]:
np.load("C:\\Users\\alexl\\Desktop\\MedCoop\\code\\readed_data\\CaseInformation.npy").shape

(415, 3, 1482, 922)

In [7]:
df_med = pd.read_excel("C:\\Users\\alexl\\Desktop\\MedCoop\\data\\result.xlsx")
df_med

,Unnamed: 0,姓名,性别,年龄,病理结果,具体结果,淋巴结转移,其他,超声分级
0,1,1周桂香,女,74.0,良,左右叶甲状腺肿,NaN,NaN,NaN
1,2,2苏丙花,女,66.0,恶,右叶乳头状癌，左叶结节性甲状腺肿伴局灶纤维化及钙化,NaN,NaN,NaN
2,3,3孙素娟,女,33.0,恶,右叶乳头状癌,NaN,NaN,NaN
3,4,4胡加福,男,70.0,良,甲状腺组织,NaN,NaN,NaN
4,5,5李永芳,女,40.0,恶,左叶中下部乳头状癌,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
410,772,279时荣平,女,54.0,恶,左叶,有,NaN,NaN
411,773,280王村,女,42.0,恶,右叶,无,NaN,NaN
412,774,281张树荣,女,67.0,良,右叶,NaN,NaN,NaN
413,775,364赵建云,女,48.0,良,双叶,NaN,NaN,NaN


In [8]:
CaseResult = df_med["病理结果"].replace({'良': 0, '恶': 1}).to_numpy().reshape([415,1])
np.save("C:\\Users\\alexl\\Desktop\\MedCoop\\code\\readed_data\\CaseResult",CaseResult)

In [9]:
np.load("C:\\Users\\alexl\\Desktop\\MedCoop\\code\\readed_data\\CaseResult.npy").shape

(415, 1)